# Notebook Corresponding to `pyspark` Notes

This notebook has the code and examples from the three sets of notes:
- `pyspark`: RDDs
- `pyspark`: pandas-on-Spark
- `pyspark`: Spark SQL

Each section should be able to be run without running the cells from the other sections.

## RDDs

First, the code from the notes is given below. Make sure that the kernel chosen to run these notes is the `pyspark` kernel. See the top right of the notebook!

Note: We can still run python code and load in libraries as normal.

Now, we start by creating our spark session.

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('my_app').config("spark.sql.ansi.enabled", "false").getOrCreate()

Let's populate a list with tuples and explicitly create an RDD.

In [15]:
#populate a list with tuples
quick_cat = lambda x: "a" if x < 20 else "b"
my_data = [(quick_cat(x), x) for x in range(1,51)]
my_data[:3]
#create the RDD
my_rdd = spark.sparkContext.parallelize(my_data)
my_rdd

ParallelCollectionRDD[32] at readRDDFromFile at PythonRDD.scala:297

This is an object stored (likely) over multiple partitions.

In [16]:
my_rdd.getNumPartitions()

128

We see that `my_rdd` doesn't actually print out the data when we look at the object. This is because there may be a ton of data and it doesn't want to show it to you by default. Instead we can perform an action like the `.take()` to actually have some data returned to us.

In [17]:
my_rdd.take(3)

[('a', 1), ('a', 2), ('a', 3)]

When we have tuple type object passed in as the data, the first value represents the `key` and the second the associated `values`.

In [18]:
my_rdd.keys().take(3)

['a', 'a', 'a']

In [19]:
my_rdd.values().take(3)

[1, 2, 3]

This allows us to do operations by key if we'd like! Note that `.count()` and `.countByKey()` are actions and so they return the value locally.

In [20]:
my_rdd.count()

50

In [21]:
my_rdd.countByKey()

defaultdict(int, {'a': 19, 'b': 31})

If instead we wanted to use the result of this counting operation as a new RDD, we could instead use something like the `mapValues()` method. This returns an RDD rather than a value and so we need to use `.collect()` to see the data.

In [22]:
my_rdd \
    .groupByKey() \
    .mapValues(len) \
    .collect()

[('a', 19), ('b', 31)]

With this, we could do some other transformation on the resulting object (say using `.map()`, which can apply a function to each element of our RDD). For instance, creating a log transformed value as well.

In [23]:
from numpy import log
my_rdd \
    .groupByKey() \
    .mapValues(len) \
    .map(lambda x: (x[0], x[1], log(x[1]))) \
    .collect()

[('a', 19, 2.9444389791664403), ('b', 31, 3.4339872044851463)]

RDD functions are hard to use though! We might want to find the total sum of the values for each key. We can use `.groupByKey()` and `mapValues()` for this but the documentation says it is better to use `aggregateByKey()`. But this function requires some confusing arguments.

In [24]:
my_rdd \
    .groupByKey() \
    .mapValues(sum) \
    .collect()

[('a', 190), ('b', 1085)]

In [25]:
my_rdd \
  .aggregateByKey(0, #initial value for each partition
                  lambda within_1, within_2: within_1 + within_2, #how to combine values on the same partition, next function is how to combine across partitions
                  lambda across_1, across_2: across_1 + across_2) \
  .collect()

[('a', 190), ('b', 1085)]

We can use `.map()` instead if we wanted to.

### MapReduce Example Done Explicity Using RDDs

Recall that in the Hadoop section, we did a MapReduce algorithm to count the number of words in Oliver Twist.  We can redo that example using Spark!  It will actually be parallelized and all that automatically across our machine too!  

In [26]:
from pyspark.sql import SparkSession
#create a spark session object 
spark = SparkSession.builder.appName('my_app').config("spark.sql.ansi.enabled", "false").getOrCreate()

Now let's read in our 53 chapters as a list.

In [28]:
my_chap = []
for i in range(1, 6):
    with open('dickens/chap' + str(i) + '.txt', 'r') as f:
        my_chap.append(f.read())

We want spark to handle this using RDDs explicitly. We can do that is via the `sparkContext.parallelize()` method.  This just tells spark to take our list and distribute it/prepare it for parallel computations.

Let's create some RDDs! We don't care about the chapters themselves, we just want the final word counts. This means we can start with an RDD without keys.

In [29]:
my_chap_rdd = spark.sparkContext.parallelize(my_chap)
type(my_chap_rdd)

pyspark.core.rdd.RDD

In [30]:
my_chap_rdd.take(1)

['chapter i  treats of the place where oliver twist was born and of the circumstances attending his birth  among other public buildings in a certain town which for many reasons it will be prudent to refrain from mentioning and to which i will assign no fictitious name there is one anciently common to most towns great or small to wit a workhouse and in this workhouse was born on a day and date which i need not trouble myself to repeat inasmuch as it can be of no possible consequence to the reader in this stage of the business at all events the item of mortality whose name is prefixed to the head of this chapter  for a long time after it was ushered into this world of sorrow and trouble by the parish surgeon it remained a matter of considerable doubt whether the child would survive to bear any name at all in which case it is somewhat more than probable that these memoirs would never have appeared or if they had that being comprised within a couple of pages they would have possessed the i

Great, now we want to take these different values (each chapter is a value) and split those strings by spaces. 

In [31]:
my_chap_rdd \
    .flatMap(lambda x: x.split(" ")) \
    .take(10)

['chapter',
 'i',
 '',
 'treats',
 'of',
 'the',
 'place',
 'where',
 'oliver',
 'twist']

Now we have an RDD whose elements are each word (again no keys). First let's filter to remove any empty spaces.

In [32]:
my_chap_rdd \
    .flatMap(lambda x: x.split(" ")) \
    .filter(lambda x: x != "") \
    .take(10)

['chapter',
 'i',
 'treats',
 'of',
 'the',
 'place',
 'where',
 'oliver',
 'twist',
 'was']

Let's do a transform on this where we make each word a key and assign it a value of 1.

In [33]:
my_chap_rdd \
    .flatMap(lambda x: x.split(" ")) \
    .filter(lambda x: x != "") \
    .map(lambda word: (word, 1)) \
    .take(5)

[('chapter', 1), ('i', 1), ('treats', 1), ('of', 1), ('the', 1)]

Nice! Now we just need to reduce this by key and we are done!

In [34]:
my_chap_rdd \
    .flatMap(lambda x: x.split(" ")) \
    .filter(lambda x: x != "") \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .take(10)

[('circumstances', 3),
 ('extant', 1),
 ('out', 26),
 ('result', 3),
 ('eight', 3),
 ('something', 6),
 ('divorce', 1),
 ('pronounced', 2),
 ('exactly', 1),
 ('step', 3)]

Let's sort this by the keys. We can sort descending by simply sorting on the negative of the values.

In [35]:
my_chap_rdd \
    .flatMap(lambda x: x.split(" ")) \
    .filter(lambda x: x != "") \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: -x[1]) \
    .take(10)

[('the', 1005),
 ('and', 439),
 ('a', 416),
 ('of', 389),
 ('to', 357),
 ('in', 257),
 ('was', 222),
 ('his', 219),
 ('he', 203),
 ('mr', 157)]

Let's collect all of the data and turn it into a regular pandas data frame.

In [36]:
results = my_chap_rdd \
    .flatMap(lambda x: x.split(" ")) \
    .filter(lambda x: x != "") \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: -x[1]) \
    .collect()
import pandas as pd
df_results = pd.DataFrame(results, columns = ["word", "count"])
df_results

,word,count
0,the,1005
1,and,439
2,a,416
3,of,389
4,to,357
...,...,...
2972,emerged,1
2973,silent,1
2974,whoever,1
2975,supporting,1


## pandas-on-Spark

Below is the code from the notes on pandas-on-Spark.

First let's import our modules.

In [32]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
ps.set_option('compute.fail_on_ansi_mode', False)

import warnings
from pyspark.pandas.utils import PandasAPIOnSparkAdviceWarningCreate a pandas-on-Spark series via `ps.Series()`

In [33]:
ps.Series([1, 3, 5, np.nan, 6, 8]) #ignore the warning

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

Create a pandas-on-Spark DataFrame via `ps.DataFrame()`

In [34]:
ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


Convert from a pandas DataFrame to a pandas-on-spark easily.

In [35]:
pdf = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/red-wine.csv", delimiter = ";")
psdf = ps.from_pandas(pdf)
psdf.head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Can subset the data using things we know like the `.loc[]` method.

In [36]:
psdf.loc[psdf.quality > 5, ["alcohol", "quality"]].head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,alcohol,quality
3,9.8,6
7,10.0,7
8,9.5,7
16,10.5,7
19,9.2,6


Can also read data directly into a pandas-on-spark data frame using the `ps.read_csv()` function (can't read from a URL though).

In [37]:
titanic_ps = ps.read_csv("data/titanic.csv") #data uploaded to jhub in data folder
titanic_ps["survived"].value_counts()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


survived
0    809
1    500
Name: count, dtype: int64

Can now do our usual summarizations using the `.groupby()` method along with a summarization method.

In [38]:
titanic_ps.groupby("survived").mean()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/groupby.py:3501: FutureWarning: Dropping invalid columns in DataFrameGroupBy.median is deprecated. In a future version, a TypeError will be raised. Before calling .median, select only columns which should be valid for the function.
  warnings.warn(
/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,pclass,age,sibsp,parch,fare,body
survived,,,,,,
1,1.962000,28.918228,0.462000,0.476000,49.361184,NaN
0,2.500618,30.545369,0.521632,0.328801,23.353831,160.809917


In [39]:
titanic_ps.describe()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


We can also use the `.transform()` and `.apply()` methods (also used in regular pandas) to perform other common operations.

First we can transform the values in our columns (say center and scale them).

In [40]:
def standardize(pser) -> ps.Series[np.float64]:
     return (pser + pser.mean())/pser.std()  # should always return the same length as input.

In [41]:
std_res = titanic_ps[["age", "fare"]] \
    .rename(columns = {"age": "o_age", "fare": "o_fare"}) \
    .join(titanic_ps[["age", "fare"]]
              .transform(standardize))
std_res.head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,o_age,o_fare,age,fare
0,29.0000,211.3375,4.085138,4.726416
1,0.9167,151.5500,2.136735,3.571295
2,2.0000,151.5500,2.211894,3.571295
3,30.0000,151.5500,4.154517,3.571295
4,25.0000,151.5500,3.807620,3.571295


In [42]:
std_res.shape

(1310, 4)

Can use `.apply()` to possible return something shorter than the original.


In [43]:
def standardize_positives(pser) -> ps.Series[np.float64]:
     return (pser[pser>30] + pser[pser>30].mean())/pser[pser>30].std()
# can return something short than input length

In [44]:
std_pos = titanic_ps[["age"]].apply(standardize_positives)
std_pos.head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,age
0,9.135889
1,10.636052
2,8.235791
3,9.635943
4,11.436139


In [45]:
std_pos.shape

(437, 1)

### MapReduce Example Done via pandas-on-Spark

Let's repeat our map reduce example but, you know, do it more easily :)

Recall the `my_chap` object.

In [2]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
ps.set_option('compute.fail_on_ansi_mode', False)
#read in each chapter to a list element
my_chap = []
for i in range(1, 6):
    with open('dickens/chap' + str(i) + '.txt', 'r') as f:
        my_chap.append(f.read())
        
my_chap[0]

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/26 18:33:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'chapter i  treats of the place where oliver twist was born and of the circumstances attending his birth  among other public buildings in a certain town which for many reasons it will be prudent to refrain from mentioning and to which i will assign no fictitious name there is one anciently common to most towns great or small to wit a workhouse and in this workhouse was born on a day and date which i need not trouble myself to repeat inasmuch as it can be of no possible consequence to the reader in this stage of the business at all events the item of mortality whose name is prefixed to the head of this chapter  for a long time after it was ushered into this world of sorrow and trouble by the parish surgeon it remained a matter of considerable doubt whether the child would survive to bear any name at all in which case it is somewhat more than probable that these memoirs would never have appeared or if they had that being comprised within a couple of pages they would have possessed the in

Let's put this into a pandas-on-Spark series and manipulate from there!

In [3]:
#combine the list elements into one large string
from functools import reduce
big_string = reduce(lambda x, y: x + y, my_chap)

#create a series with the big string
chap_pss = ps.Series(big_string)
chap_pss

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


0    chapter i  treats of the place where oliver tw...
dtype: object

Now use the `.str.split()` method on a pandas-on-Spark series to create a series with a list of words.

In [4]:
chap_pss.str.split()

0    [chapter, i, treats, of, the, place, where, ol...
dtype: object

What we want is to have each word be in a column with the count associated as another column. Let's convert the list stored in the series to a data frame. Then remove the empty spaces.

In [5]:
word_df = ps.DataFrame(chap_pss.str.split(" ")[0], columns = ["word"])
word_df.head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,word
0,chapter
1,i
2,
3,treats
4,of


In [6]:
word_df = word_df.loc[word_df.word != ""]
word_df.head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,word
0,chapter
1,i
3,treats
4,of
5,the


At first, we can just assign each value to a 1 and then use our usual `.groupby()` to get our desired result. Add a count of 1 for each word.

In [7]:
word_df["count"] = 1
word_df.head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,word,count
0,chapter,1
1,i,1
3,treats,1
4,of,1
5,the,1


Awesome! Now just group by the word and sum it up!

In [8]:
word_df \
    .groupby("word") \
    .sum() \
    .head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,count
word,
circumstances,3
among,5
day,13
buildings,1
name,8


Sort it so we can compare to our previous work.

In [9]:
word_df \
    .groupby("word") \
    .sum() \
    .sort_values(by = "count", ascending = False) \
    .head()

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,count
word,
the,1005
and,439
a,416
of,389
to,357


Woot!

## Spark SQL

Below is the code from the Spark SQL notes.

Start with creation a spark session.

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('my_app').config("spark.sql.ansi.enabled", "false").getOrCreate()

26/02/26 18:34:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Now let's look at a few ways to create a Spark SQL Data Frame.

In [11]:
from pyspark.sql import Row
from datetime import datetime, date
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [12]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [13]:
df = spark.read.load("data/neuralgia.csv",
                     format="csv", 
                     sep=",", 
                     inferSchema="true", 
                     header="true")
df

DataFrame[Treatment: string, Sex: string, Age: int, Duration: int, Pain: string]

In [14]:
import pandas as pd
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

Conveniently, we can go back and forth between Spark SQL style Data Frames and pandas-on-Spark style Data Frames.

In [15]:
sdf = spark.read.load("data/neuralgia.csv",
                     format="csv", 
                     sep=",", 
                     inferSchema="true", 
                     header="true")
type(sdf)

pyspark.sql.classic.dataframe.DataFrame

In [16]:
dfps = sdf.pandas_api()
type(dfps)

pyspark.pandas.frame.DataFrame

In [17]:
sdf2 = dfps.to_spark()
type(sdf2)

/opt/tljh/user/envs/pySpark3/lib/python3.9/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


pyspark.sql.classic.dataframe.DataFrame

Schema and column names are important to know.

In [18]:
df = spark.read.load("data/neuralgia.csv",
                     format="csv", 
                     sep=",", 
                     inferSchema="true", 
                     header="true")
df.printSchema()
df.columns

root
 |-- Treatment: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Pain: string (nullable = true)



['Treatment', 'Sex', 'Age', 'Duration', 'Pain']

We can return the data using `.take()`, `.show()`. and `.collect()`.

In [19]:
df.show(3)

+---------+---+---+--------+----+
|Treatment|Sex|Age|Duration|Pain|
+---------+---+---+--------+----+
|        P|  F| 68|       1|  No|
|        B|  M| 74|      16|  No|
|        P|  F| 67|      30|  No|
+---------+---+---+--------+----+
only showing top 3 rows


In [20]:
df.take(3)

[Row(Treatment='P', Sex='F', Age=68, Duration=1, Pain='No'),
 Row(Treatment='B', Sex='M', Age=74, Duration=16, Pain='No'),
 Row(Treatment='P', Sex='F', Age=67, Duration=30, Pain='No')]

Next, we'll look at common transformations. Starting with column operations.

In [21]:
#select columns you want
df.select("Age")

DataFrame[Age: int]

In [22]:
df.Age

Column<'Age'>

In [23]:
df.select("Age", "Pain").show(3)

+---+----+
|Age|Pain|
+---+----+
| 68|  No|
| 74|  No|
| 67|  No|
+---+----+
only showing top 3 rows


`.withColumn()` can be used to create new columns.

In [24]:
df.withColumn("Current_Age", df.Age + 2).show(3)

+---------+---+---+--------+----+-----------+
|Treatment|Sex|Age|Duration|Pain|Current_Age|
+---------+---+---+--------+----+-----------+
|        P|  F| 68|       1|  No|         70|
|        B|  M| 74|      16|  No|         76|
|        P|  F| 67|      30|  No|         69|
+---------+---+---+--------+----+-----------+
only showing top 3 rows


Can also rename columns.

In [25]:
from pyspark.sql.functions import col
df \
  .withColumnRenamed('Age', 'Former_Age') \
  .withColumn("Current_Age", col("Former_Age") + 2) \
  .show(3)

+---------+---+----------+--------+----+-----------+
|Treatment|Sex|Former_Age|Duration|Pain|Current_Age|
+---------+---+----------+--------+----+-----------+
|        P|  F|        68|       1|  No|         70|
|        B|  M|        74|      16|  No|         76|
|        P|  F|        67|      30|  No|         69|
+---------+---+----------+--------+----+-----------+
only showing top 3 rows


We can using conditional logic with `when()` from `pyspark.sql.functions`.

In [26]:
from pyspark.sql.functions import *
df.withColumn("Age_cat", 
               when(df.Age>75, "75+")
              .when(df.Age>=70, "70-75")
              .otherwise("<70")) \
    .show(3)

+---------+---+---+--------+----+-------+
|Treatment|Sex|Age|Duration|Pain|Age_cat|
+---------+---+---+--------+----+-------+
|        P|  F| 68|       1|  No|    <70|
|        B|  M| 74|      16|  No|  70-75|
|        P|  F| 67|      30|  No|    <70|
+---------+---+---+--------+----+-------+
only showing top 3 rows


In [27]:
df.withColumn("Age_cat", 
               when(df.Age>75, "75+")
              .when(df.Age>=70, "70-75")
              .otherwise("<70")) \
   .withColumn("ln_Duration", log(df.Duration)) \
   .show(3)

+---------+---+---+--------+----+-------+------------------+
|Treatment|Sex|Age|Duration|Pain|Age_cat|       ln_Duration|
+---------+---+---+--------+----+-------+------------------+
|        P|  F| 68|       1|  No|    <70|               0.0|
|        B|  M| 74|      16|  No|  70-75| 2.772588722239781|
|        P|  F| 67|      30|  No|    <70|3.4011973816621555|
+---------+---+---+--------+----+-------+------------------+
only showing top 3 rows


We can also create our own functions with `udf`.

In [28]:
code_trt = udf(lambda x: "P Trt" if x == "P" else "Other")
df.withColumn('my_trt', code_trt('Treatment')).show(3)

+---------+---+---+--------+----+------+
|Treatment|Sex|Age|Duration|Pain|my_trt|
+---------+---+---+--------+----+------+
|        P|  F| 68|       1|  No| P Trt|
|        B|  M| 74|      16|  No| Other|
|        P|  F| 67|      30|  No| P Trt|
+---------+---+---+--------+----+------+
only showing top 3 rows


We can do the common operations on rows as well.

In [29]:
df.sort(df.Duration).show(3)

+---------+---+---+--------+----+
|Treatment|Sex|Age|Duration|Pain|
+---------+---+---+--------+----+
|        A|  M| 69|       1|  No|
|        B|  M| 70|       1|  No|
|        B|  F| 78|       1|  No|
+---------+---+---+--------+----+
only showing top 3 rows


In [30]:
df.sort(df.Duration, ascending = False).show(3)

+---------+---+---+--------+----+
|Treatment|Sex|Age|Duration|Pain|
+---------+---+---+--------+----+
|        B|  F| 72|      50|  No|
|        A|  M| 62|      42|  No|
|        B|  F| 69|      42|  No|
+---------+---+---+--------+----+
only showing top 3 rows


In [31]:
df.filter(df.Age < 65).show(3)

+---------+---+---+--------+----+
|Treatment|Sex|Age|Duration|Pain|
+---------+---+---+--------+----+
|        A|  F| 63|      27|  No|
|        A|  M| 62|      42|  No|
|        P|  F| 64|       1| Yes|
+---------+---+---+--------+----+
only showing top 3 rows


We can do basic summaries including grouped summaries!

In [32]:
df.select("Age", "Pain").describe().show()

+-------+-----------------+----+
|summary|              Age|Pain|
+-------+-----------------+----+
|  count|               60|  60|
|   mean|            70.05|NULL|
| stddev|5.189379637003748|NULL|
|    min|               59|  No|
|    max|               83| Yes|
+-------+-----------------+----+



In [33]:
df \
    .select(["Duration", "Age", "Treatment"]) \
    .agg(sum("Duration"), avg("Age"), count("Treatment")) \
    .show()

+-------------+--------+----------------+
|sum(Duration)|avg(Age)|count(Treatment)|
+-------------+--------+----------------+
|         1004|   70.05|              60|
+-------------+--------+----------------+



In [34]:
df.select(["Duration", "Age", "Treatment"]) \
    .groupBy("Treatment") \
    .sum() \
    .withColumnRenamed("sum(Duration)", "sum_Duration") \
    .withColumnRenamed("sum(Age)", "sum_Age") \
    .show()

+---------+------------+-------+
|Treatment|sum_Duration|sum_Age|
+---------+------------+-------+
|        B|         386|   1417|
|        A|         327|   1385|
|        P|         291|   1401|
+---------+------------+-------+



In [35]:
df.createTempView("df")
spark.sql("SELECT sex, age FROM df LIMIT 4").show()

+---+---+
|sex|age|
+---+---+
|  F| 68|
|  M| 74|
|  F| 67|
|  M| 66|
+---+---+



### MapReduce Example Done via Spark SQL 

Let's redo it with Spark SQL!

In [36]:
#read in each chapter to a list element
my_chap = []
for i in range(1, 6):
    with open('dickens/chap' + str(i) + '.txt', 'r') as f:
        my_chap.append(f.read())

from pyspark.sql.types import StringType
sql_text = spark.createDataFrame(my_chap, StringType())
sql_text

DataFrame[value: string]

In [37]:
sql_text.take(1)

[Row(value='chapter i  treats of the place where oliver twist was born and of the circumstances attending his birth  among other public buildings in a certain town which for many reasons it will be prudent to refrain from mentioning and to which i will assign no fictitious name there is one anciently common to most towns great or small to wit a workhouse and in this workhouse was born on a day and date which i need not trouble myself to repeat inasmuch as it can be of no possible consequence to the reader in this stage of the business at all events the item of mortality whose name is prefixed to the head of this chapter  for a long time after it was ushered into this world of sorrow and trouble by the parish surgeon it remained a matter of considerable doubt whether the child would survive to bear any name at all in which case it is somewhat more than probable that these memoirs would never have appeared or if they had that being comprised within a couple of pages they would have posse

Ok, first we need to split the words out within each *row*. When we read in all the SQL functions there was a `split()` function that will work for us!

Note the way we use the function without `.withColumn()` by using `.select()`. This is a common way to use these functions without adding to the original data frame.

In [38]:
from pyspark.sql.functions import *

In [39]:
split(sql_text.value, " ").alias("words")

Column<'split(value, ' ', -1) AS words'>

In [40]:
sql_text.select(split(sql_text.value, " ").alias("words")).show(4)

+--------------------+
|               words|
+--------------------+
|[chapter, i, , tr...|
|[chapter, ii, , t...|
|[chapter, iii, , ...|
|[chapter, iv, , o...|
+--------------------+
only showing top 4 rows


Ok, now we have a data frame with one column where each entry is a list of the words! This is closer. 

What we need to do is now **explode** out these lists. We read in a function called **explode** that will split these values up and create new rows for each entry!

Notice how we call the function inside select again.

In [41]:
explode(split(sql_text.value, " ")).alias("word")

Column<'explode(split(value, ' ', -1)) AS word'>

In [42]:
sql_text.select(explode(split(sql_text.value, " ")).alias("word")).show(4)

+-------+
|   word|
+-------+
|chapter|
|      i|
|       |
| treats|
+-------+
only showing top 4 rows


Woo, almost there. Now we can filter out the blank spaces.

In [43]:
my_words = sql_text.select(explode(split(sql_text.value, " ")).alias("word"))
my_words

DataFrame[word: string]

In [44]:
my_words \
    .filter(my_words.word != "") \
    .show(4)

+-------+
|   word|
+-------+
|chapter|
|      i|
| treats|
|     of|
+-------+
only showing top 4 rows


Finally we group and count!

In [45]:
my_words \
    .filter(my_words.word != "") \
    .groupBy("word") \
    .count() \
    .show(5)

+----------+-----+
|      word|count|
+----------+-----+
|      some|   31|
|       few|    7|
|      hope|    5|
| overseers|    2|
|surrounded|    2|
+----------+-----+
only showing top 5 rows


Arrange it!

In [46]:
counts = my_words \
                .filter(my_words.word != "") \
                .groupBy("word") \
                .count()
counts.sort(counts["count"], ascending = False).show(10)

+----+-----+
|word|count|
+----+-----+
| the| 1005|
| and|  439|
|   a|  416|
|  of|  389|
|  to|  357|
|  in|  257|
| was|  222|
| his|  219|
|  he|  203|
|  mr|  157|
+----+-----+
only showing top 10 rows
